# 데이터 다운 및 초기 탐색

In [6]:
import os
import json
import glob
from collections import defaultdict
from tqdm import tqdm  # 진행 상황 표시

# Windows에서는 경로 앞에 r을 붙여야 백슬래시 문제를 피할 수 있다
base_dir = r"C:\Users\daboi\OneDrive\Desktop\ai05-level1-project"

train_img_dir = os.path.join(base_dir, "train_images")
test_img_dir = os.path.join(base_dir, "test_images")
train_ann_dir = os.path.join(base_dir, "train_annotations")

# train_annotations 폴더 및 모든 하위 폴더에서 .json 파일 검색
json_files = glob.glob(os.path.join(train_ann_dir, "**", "*.json"), recursive = True)

print(f"총 {len(json_files)}개의 JSON 어노테이션 파일")
print(f"총 {len(os.listdir(train_img_dir))}개의 학습 이미지 파일")
print(f"총 {len(os.listdir(test_img_dir))}개의 테스트 이미지 파일")

총 4526개의 JSON 어노테이션 파일
총 1489개의 학습 이미지 파일
총 843개의 테스트 이미지 파일


- 4526개의 어노테이션이 있다 학습 이미지 파일은 1489개이다 이미지 하나에 여러 알약이 있다는 뜻이겠다(이미지를 직접 해봐도 대부분 3 - 4개씩 있었다)
- 테스트 이미지는 원래 어노테이션이 없다

In [ ]:
# JSON 파일들을 읽어 하나의 마스터 데이터로 통합
master_data = defaultdict(lambda: {'image_path': '', 'annotations': []})
class_to_id = {}
current_id = 0

for json_path in tqdm(json_files, desc="어노테이션 통합 중"):
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # JSON 구조 확인 (images, annotations)
        img_info = data['images'][0]
        img_filename = img_info['file_name']

        # 이미지 경로 설정
        if not master_data[img_filename]['image_path']:
            image_path = os.path.join(train_img_dir, img_filename)
            if not os.path.isfile(image_path):
                print(f"경고: 이미지 파일이 존재하지 않습니다 - {image_path}")
            master_data[img_filename]['image_path'] = image_path

        # 여러 개의 어노테이션 처리
        for ann in data['annotations']:
            bbox = ann['bbox']  # [x_min, y_min, w, h] 형식

            # 클래스명 위치 확인
            if 'dl_name' in ann:
                class_name = ann['dl_name']
            else:
                class_name = img_info.get('dl_name', 'Unknown')

            # 새로운 클래스면 ID 부여
            if class_name not in class_to_id:
                class_to_id[class_name] = current_id
                current_id += 1

            class_id = class_to_id[class_name]

            # 어노테이션 추가
            master_data[img_filename]['annotations'].append({
                'class_id': class_id,
                'bbox': bbox
            })

    except Exception as e:
        print(f"Error processing file {json_path}: {e}")

# 결과 저장
with open(os.path.join(base_dir, "train_master_annotations.json"), "w", encoding = 'utf-8') as f:
    json.dump(master_data, f, ensure_ascii = False, indent = 4)

with open(os.path.join(base_dir, "class_to_id.json"), "w", encoding = 'utf-8') as f:
    json.dump(class_to_id, f, ensure_ascii = False, indent = 4)

print("-" * 30)
print(f"통합 완료 총 {len(master_data)}개의 이미지 데이터를 처리")
print(f"총 {len(class_to_id)}개의 고유 클래스(알약 종류)를 발견")

어노테이션 통합 중: 100%|██████████| 4526/4526 [00:00<00:00, 10857.55it/s]

------------------------------
통합 완료 총 1489개의 이미지 데이터를 처리
총 73개의 고유 클래스(알약 종류)를 발견


# 데이터 분석

In [8]:
folder = r"C:\Users\daboi\OneDrive\Desktop\ai05-level1-project\train_annotations\K-003483-022347-027777-036637_json\K-003483"
print(os.listdir(folder))

['K-003483-022347-027777-036637_0_2_0_2_70_000_200.json', 'K-003483-022347-027777-036637_0_2_0_2_90_000_200.json']


JSON 파일은 COCO 데이터셋 형식이다

In [ ]:
sample_json = r"C:\Users\daboi\OneDrive\Desktop\ai05-level1-project\train_annotations\K-003483-022347-027777-036637_json\K-003483\K-003483-022347-027777-036637_0_2_0_2_70_000_200.json"

with open(sample_json, "r", encoding = "utf-8") as f:
    data = json.load(f)

# 샘플 구조 확인
print("최상위 키:", data.keys())
print("\nimages 예시:", data["images"][0])
print("\nannotations 예시:", data["annotations"][0])

최상위 키: dict_keys(['images', 'type', 'annotations', 'categories'])

images 예시: {'file_name': 'K-003483-022347-027777-036637_0_2_0_2_70_000_200.png', 'width': 976, 'height': 1280, 'imgfile': 'K-003483-022347-027777-036637_0_2_0_2_70_000_200.png', 'drug_N': 'K-003483', 'drug_S': '정상알약', 'back_color': '연회색 배경', 'drug_dir': '앞면', 'light_color': '주백색', 'camera_la': 70, 'camera_lo': 0, 'size': 200, 'dl_idx': '3482', 'dl_mapping_code': 'K-003483', 'dl_name': '기넥신에프정(은행엽엑스)(수출용)', 'dl_name_en': 'Ginexin-F Tab.', 'img_key': 'http://connectdi.com/design/img/drug/148609543321800170.jpg', 'dl_material': '은행엽건조엑스', 'dl_material_en': 'Ginkgo Leaf Ext.', 'dl_custom_shape': '정제, 저작정', 'dl_company': '에스케이케미칼(주)', 'dl_company_en': 'Sk Chemicals', 'di_company_mf': '', 'di_company_mf_en': '', 'item_seq': 199102014, 'di_item_permit_date': '19910906', 'di_class_no': '[02190]기타의 순환계용약', 'di_etc_otc_code': '일반의약품', 'di_edi_code': '644700130,A18901361', 'chart': '진녹색의 타원형 필름 코팅정', 'drug_shape': '장방형', 'thick': 

## jason 파일 구조
    ### JSON 데이터 구조 정리

```yaml
data (dict)
│
├── "images" (list)
│     └── [0] (dict)
│          ├── file_name: 이미지 파일명 (예: "K-001900-010224-016551-031705.png")
│          ├── width: 이미지 너비 (픽셀 단위)
│          ├── height: 이미지 높이 (픽셀 단위)
│          ├── imgfile: 이미지 파일명 (중복 정보)
│          ├── drug_N: 약물 코드
│          ├── drug_S: 약물 상태 설명 (예: "정상알약")
│          ├── back_color: 배경색
│          ├── drug_dir: 앞/뒷면 여부
│          ├── light_color: 촬영 조명 색
│          ├── camera_la: 카메라 각도 (위치/각도 관련)
│          ├── camera_lo: 카메라 방향
│          ├── size: 이미지 크기
│          ├── dl_idx: 데이터셋 내부 약물 ID
│          ├── dl_mapping_code: 약물 코드 (중복)
│          ├── dl_name: 약 이름 (국문)
│          ├── dl_name_en: 약 이름 (영문)
│          ├── img_key: 약 이미지 URL
│          ├── dl_material: 성분명 (국문)
│          ├── dl_material_en: 성분명 (영문)
│          ├── dl_custom_shape: 제형 정보 (정제, 저작정 등)
│          ├── dl_company: 제조사 (국문)
│          ├── dl_company_en: 제조사 (영문)
│          ├── item_seq: 품목일련번호
│          ├── di_item_permit_date: 허가일자
│          ├── di_class_no: 의약품 분류 번호
│          ├── di_etc_otc_code: 전문/일반 여부
│          ├── chart: 약 설명
│          ├── drug_shape: 알약 모양
│          ├── thick: 두께
│          ├── leng_long: 장축 길이
│          ├── leng_short: 단축 길이
│          ├── print_front: 전면 인쇄 문자
│          ├── print_back: 후면 인쇄 문자
│          ├── color_class1: 주요 색상
│          ├── img_regist_ts: 이미지 등록일
│          ├── form_code_name: 제형 코드명
│          ├── change_date: 데이터 갱신일
│          └── id: 이미지 고유 ID
│
├── "type" (str)
│     └── 어노테이션 타입 (예: "instances")
│
├── "annotations" (list)
│     └── [0] (dict)
│          ├── area: 바운딩 박스 영역 크기
│          ├── iscrowd: 군집 여부 (0=단일 객체)
│          ├── bbox: [x, y, w, h] (바운딩 박스 좌표)
│          ├── category_id: 카테고리 ID
│          ├── ignore: 무시 여부
│          ├── id: 어노테이션 ID
│          └── image_id: 이미지 ID
│
└── "categories" (list)
      └── [0] (dict)
           ├── supercategory: 상위 카테고리 (pill)
           ├── id: 카테고리 ID
           └── name: 카테고리 이름 (국문 약명)
```

In [11]:
# 모든 JSON 파일 수집
json_files = glob.glob(os.path.join(train_ann_dir, "**", "*.json"), recursive=True)

# 이미지별 어노테이션 개수 카운트
img_to_ann_count = defaultdict(int)

for json_path in tqdm(json_files, desc="매칭 검증 중"):
    try:
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        img_info = data["images"][0]
        img_filename = img_info["file_name"]

        img_to_ann_count[img_filename] += len(data["annotations"])

    except Exception as e:
        print(f"Error in {json_path}: {e}")

print(f"총 {len(img_to_ann_count)}개의 이미지가 어노테이션과 매칭")

# 어노테이션 없는 이미지 찾기
train_imgs = set(os.listdir(train_img_dir))
ann_imgs = set(img_to_ann_count.keys())

no_ann_imgs = train_imgs - ann_imgs
print(f"어노테이션이 없는 이미지 수: {len(no_ann_imgs)}")

# 이미지당 평균 어노테이션 개수
avg_ann = sum(img_to_ann_count.values()) / len(img_to_ann_count)
print(f"이미지당 평균 어노테이션 개수: {avg_ann:.2f}")

# 샘플 출력
for img, count in list(img_to_ann_count.items())[:10]:
    print(img, ":", count, "개 어노테이션")

매칭 검증 중: 100%|██████████| 4526/4526 [00:00<00:00, 12767.56it/s]

총 1489개의 이미지가 어노테이션과 매칭
어노테이션이 없는 이미지 수: 0
이미지당 평균 어노테이션 개수: 3.04
K-001900-010224-016551-031705_0_2_0_2_70_000_200.png : 3 개 어노테이션
K-001900-010224-016551-031705_0_2_0_2_75_000_200.png : 2 개 어노테이션
K-001900-010224-016551-031705_0_2_0_2_90_000_200.png : 3 개 어노테이션
K-001900-010224-016551-033009_0_2_0_2_70_000_200.png : 2 개 어노테이션
K-001900-010224-016551-033009_0_2_0_2_75_000_200.png : 2 개 어노테이션
K-001900-010224-016551-033009_0_2_0_2_90_000_200.png : 3 개 어노테이션
K-001900-016548-018110-021026_0_2_0_2_70_000_200.png : 4 개 어노테이션
K-001900-016548-018110-021026_0_2_0_2_75_000_200.png : 4 개 어노테이션
K-001900-016548-018110-021026_0_2_0_2_90_000_200.png : 2 개 어노테이션
K-001900-016548-018110-027926_0_2_0_2_70_000_200.png : 3 개 어노테이션
